In [0]:
%pip install pyomo --quiet
dbutils.library.restartPython()

In [0]:
import os
import mlflow
import numpy as np
import pandas as pd
import pyomo.environ as pyo

In [0]:
%sh bash install_cbc.sh

The code block below:

Builds the three-tier network exactly with “close” Tier 3 → Tier 2 links (each Tier-3 node picks the nearest Tier-2 node, with a random tie-break when it sits between two). Guarantees no isolation – every Tier-1 node receives ≥ 1 edge from Tier-2, and every Tier-2 node receives ≥ 1 edge from Tier-3 (the assertion would fail otherwise). Visualises the tiers cleanly with distinctive shapes (●, ■, ▲) and a layered layout.

In [0]:
# ---------------------------------------------------------------
# Builds a synthetic 3-tier network dataset for optimisation.
# ---------------------------------------------------------------

import random, string, math
import matplotlib.pyplot as plt
from itertools import product
from collections import defaultdict

# ------------- 0 — reproduce the exact network ------------------
N1, N2, N3 = 5, 10, 20
#N1, N2, N3 = 50, 100, 200
assert N3 == 2 * N2
random.seed(777)                 # <- keep topology reproducible

tier1 = [f"T1_{i}" for i in range(1, N1 + 1)]
tier2 = [f"T2_{i}" for i in range(1, N2 + 1)]
tier3 = [f"T3_{i}" for i in range(1, N3 + 1)]

edges = []                         # (src, tgt)

# ---- Tier-2 → Tier-1  (1–3 edges out of each Tier-2) ----------
t2_out = {t2: set() for t2 in tier2}
shuffled_t2 = tier2.copy()
random.shuffle(shuffled_t2)
for t1_node, t2_node in zip(tier1, shuffled_t2):
    edges.append((t2_node, t1_node))
    t2_out[t2_node].add(t1_node)

for t2_node in tier2:
    desired = random.randint(1, 3)
    while len(t2_out[t2_node]) < desired:
        candidate = random.choice(tier1)
        if candidate not in t2_out[t2_node]:
            edges.append((t2_node, candidate))
            t2_out[t2_node].add(candidate)

# ---- Tier-3 → Tier-2  (exactly 1 edge per Tier-3) -------------
incoming_t2 = {t2: 0 for t2 in tier2}

tier3_even = tier3[::2]
for idx, t3_node in enumerate(tier3_even):
    tgt = tier2[idx]
    edges.append((t3_node, tgt))
    incoming_t2[tgt] += 1

tier3_odd = tier3[1::2]
for n, t3_node in enumerate(tier3_odd):
    low  = n
    high = min(n + 1, N2 - 1)
    tgt  = tier2[random.choice([low, high])] if low != high else tier2[low]
    edges.append((t3_node, tgt))
    incoming_t2[tgt] += 1

# ------------- 1 — part types & supplier part types ---------------------
# Generic part codes a…z.  Each tier2 and tier3 node produces ONE part type.
n = math.ceil(len(tier2)/3) + math.ceil(len(tier3)/2)
part_types = list(map(''.join, product(string.ascii_lowercase, repeat=3)))[:n]  # ['aaa','aab','aac',...]

supplier_part_type = {}

# 3 adjacent tier2 nodes produce the same part type.
for idx, node in enumerate(tier2):
  supplier_part_type[node] = part_types[math.floor(idx/3)]

# 2 adjacent tier3 nodes produce the same part type
for idx, node in enumerate(tier3):
  supplier_part_type[node] = part_types[math.ceil(len(tier2)/3) + math.floor(idx/2)]

# ------------- 2 — nested sets  N_minus , N_plus , P -----------
N_plus = defaultdict(set)          # i → list_of_children j
N_minus = defaultdict(set)         # j → list_of_part_types k
P = defaultdict(list)               # (j,k) → list_of_parents i

for i, j in edges:
    if j in tier1 + tier2:
        N_minus[j].add(supplier_part_type[i])
N_minus = {node: sorted(list(parts)) for node, parts in N_minus.items()}

for i, j in edges:
    if i in tier2 + tier3:
        N_plus[i].add(j)
N_plus = {node: sorted(list(childs)) for node, childs in N_plus.items()}

# parent nodes of node j of part type k: dict  (j,k) ↦ list_of_i  (𝒫_{jk})
for i, j in edges:
    if j in tier1 + tier2:
        P[(j, supplier_part_type[i])].append(i)

# ------------- 3 — scalar & tabular parameters ------------------
rng_int   = lambda lo, hi: random.randint(lo, hi)
rng_float = lambda lo, hi, r=2: round(random.uniform(lo, hi), r)

# Profit margin for finished products
f = {j: rng_float(0.05, 0.30) for j in tier1}

# On-hand inventory for every node
s = {n: rng_int(400, 1800) for n in tier1 + tier2 + tier3}

# Demand per TTR for finished products
d = {j: rng_int(400, 1800) for j in tier1}

# Production capacity per TTR for every node
c = {n: rng_int(500, 2500) for n in tier1 + tier2 + tier3}

# Time-to-recover for this disruption scenario
t = 1

# A small share of Tier-2 + Tier-3 nodes disrupted
disrupted_count = max(1, int(0.10 * (len(tier2) + len(tier3))))
disrupted       = random.sample(tier2 + tier3, disrupted_count)

# ------------- 4 — quick smoke test when run directly ----------
if __name__ == "__main__":
    print("Tier sizes  :", len(tier1), len(tier2), len(tier3))
    print("Edges       :", len(edges))
    print("Disrupted   :", disrupted)
    print("------------------------------------------------------")
    print("f:", {j: f[j] for j in tier1})
    print("d:", {j: d[j] for j in tier1})
    print("s:", {j: s[j] for j in tier1})
    print("c:", {j: c[j] for j in tier2})
    print("part_types:", part_types)
    print("------------------------------------------------------")
    print(f"N_minus:", {j: N_minus[j] for j in tier1})
    print(f"N_plus:", {j: N_plus[j] for j in tier2})
    print(f"P:", P)
# ----------------------------------------------------------------
# 5 — objects exported for optimisation_model.py
#     (import dataset_from_network as ds; ds.tier1, ds.N_minus, …)
# ----------------------------------------------------------------
__all__ = [
    # elementary sets
    'tier1', 'tier2', 'tier3', 'part_types', 'disrupted',
    # nested sets
    'N_minus', 'N_plus', 'P',
    # parameters
    'f', 's', 't', 'd', 'c',
]

In [0]:
# 1) one distinct colour per *code* in the dict
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

codes       = sorted(set(supplier_part_type.values()))
cmap        = plt.get_cmap("tab20", len(codes))
code_colour = {code: cmap(i) for i, code in enumerate(codes)}

# Fallback for Tier-1 (or any node not in the dict)
default_colour = "#9e9e9e"        # mid-grey

# Helper that returns a list of colours in node order
def colours_for(nodes):
    return [
        code_colour.get(supplier_part_type.get(n, None), default_colour)
        for n in nodes
    ]

# -------------------------------------------------------------
# POSITIONS (keep the centred, tier-specific gaps version)
# -------------------------------------------------------------
pos = {}

gap_t1 = 3.0      # widest spacing
gap_t2 = 2.2      # medium spacing
gap_t3 = 1.5      # default spacing

tier_specs = [            # (nodes , gap , y-coordinate)
    (tier1, gap_t1, 2),
    (tier2, gap_t2, 1),
    (tier3, gap_t3, 0),
]

# 1) largest physical width among tiers (needed for centring)
max_width = max((len(nodes) - 1) * gap for nodes, gap, _ in tier_specs)

# 2) place each node
for nodes, gap, y in tier_specs:
    width     = (len(nodes) - 1) * gap
    x_offset  = (max_width - width) / 2        # shift so tier is centred
    for idx, node in enumerate(nodes):
        pos[node] = (x_offset + idx * gap, y)

# -------------------------------------------------------------
# VISUALISATION
# -------------------------------------------------------------
fig, ax = plt.subplots(figsize=(15, 6))

# Tier-1 (neutral grey)
ax.scatter([pos[n][0] for n in tier1], [pos[n][1] for n in tier1],
           s=550, marker='o', c=colours_for(tier1),
           edgecolor='k', linewidth=0.5, label="Tier 1 (products)")

# Tier-2 (coloured by part-type)
ax.scatter([pos[n][0] for n in tier2], [pos[n][1] for n in tier2],
           s=550, marker='s', c=colours_for(tier2),
           edgecolor='k', linewidth=0.5, label="Tier 2 (sub-assemblies)")

# Tier-3 (coloured by part-type)
ax.scatter([pos[n][0] for n in tier3], [pos[n][1] for n in tier3],
           s=450, marker='^', c=colours_for(tier3),
           edgecolor='k', linewidth=0.5, label="Tier 3 (suppliers)")

# Node labels
for node, (x, y) in pos.items():
    ax.text(x, y, node, ha='center', va='center', fontsize=8)

# Directed edges
for src, tgt in edges:
    sx, sy = pos[src]
    tx, ty = pos[tgt]
    ax.annotate("",
                xy=(tx, ty), xytext=(sx, sy),
                arrowprops=dict(arrowstyle="-|>", lw=0.8))

# Axes & title
max_width = max((len(tier1) - 1) * 3.0, (len(tier2) - 1) * 2.2, (len(tier3) - 1) * 1.5)
ax.set_xlim(-3.0, max_width + 3.0)
ax.set_ylim(-0.7, 2.7)
ax.axis("off")
plt.title("Three-Tier Directed Network\n(coloured by supplier_part_type)")

# Custom legend: one patch per part-type code
patches = [mpatches.Patch(color=code_colour[c], label=c) for c in codes]
first_legend = ax.legend(handles=patches, title="supplier_part_type", fontsize=8,
                         title_fontsize=9, loc="upper left", bbox_to_anchor=(1.02, 1))
# Add the tier legend underneath
ax.legend(loc="upper left")
ax.add_artist(first_legend)       # keep both legends

plt.tight_layout()
plt.show()

In [0]:
# optimisation_model.py
from pyomo.environ import *

# ------------------------------------------------------------------
# 1.  Prepare your data  (⇦ replace the dummy placeholders)
# ------------------------------------------------------------------
data = {
    # elementary sets ------------------------------------------------
    'V'      : tier1,               # product nodes
    'D'      : tier1 + tier2,       # all BUT leaf nodes
    'U'      : tier2 + tier3,       # all BUT product nodes
    'K'      : part_types,          # part types  (k ∈ 𝒩⁻(j))
    'S'      : disrupted,           # disrupted nodes in scenario n
    # nested sets ----------------------------------------------------
    'N_minus' : N_minus,         # parts required to produce node j: dict  j ↦ list_of_k   (𝒩⁻(j))
    'N_plus'  : N_plus,          # child nodes of node i: dict  i ↦ list_of_j   (𝒩⁺(i))
    'P'       : P,               # parent nodes of node j of part type k: dict  (j,k) ↦ list_of_i  (𝒫_{jk})
    # parameters -----------------------------------------------------
    'f'  : f,    # profit margin of 1 unit of j
    's'  : s,    # finished-goods inventory of i
    't'  : t,    # TTR for disruption scenario n   (a scalar)
    'd'  : d,    # demand for j per TTR
    'c'  : c,    # plant capacity per TTR
}


# ------------------------------------------------------------------
# 2.  Build the ConcreteModel
# ------------------------------------------------------------------

m = ConcreteModel()

# ---- 2.1  sets ----------------------------------------------------
m.V   = Set(initialize=data['V'])
m.D   = Set(initialize=data['D'])
m.U   = Set(initialize=data['U'])
m.K   = Set(initialize=data['K'])
m.S   = Set(initialize=data['S'])

m.N_minus = Set(m.D, initialize=lambda mdl, j: data['N_minus'][j])
m.N_plus  = Set(m.U, initialize=lambda mdl, i: data['N_plus'][i])

# handy union of *all* nodes that may carry production volume
m.NODES = pyo.Set(initialize=list(
        set(data['V']) | set(data['U'])
    ))

# 𝒫_{jk} as map (j,k) → list_of_i
m.P = Set(dimen=3, initialize=[
        (i, j, k)
        for (j, k), I in data['P'].items()
        for i in I
    ])

# ---- 2.2  parameters ---------------------------------------------
m.f = Param(m.V, initialize=data['f'], within=NonNegativeReals)          # impact (profit margin)
m.s = Param(m.NODES, initialize=data['s'], within=NonNegativeIntegers)
m.t = Param(initialize=data['t'], within=PositiveReals)
m.d = Param(m.V, initialize=data['d'], within=NonNegativeIntegers)
m.c = Param(m.NODES, initialize=data['c'], within=NonNegativeIntegers)

# ---- 2.3  decision variables -------------------------------------
m.u = Var(m.NODES, domain=NonNegativeIntegers)      # production quantity of node i
m.l = Var(m.V, domain=NonNegativeIntegers)          # lost volume of product j
# y is only needed for (i,j) pairs that actually make sense
m.y_index = Set(within=m.U * m.NODES, initialize=lambda mdl: [
        (i, j) for i in mdl.U for j in mdl.N_plus[i]
    ])
m.y = Var(m.y_index, domain=NonNegativeIntegers)


# ------------------------------------------------------------------
# 3.  objective
# ------------------------------------------------------------------

def obj_rule(mdl):
    return sum(mdl.f[j] * mdl.l[j] for j in mdl.V)
m.OBJ = Objective(rule=obj_rule, sense=minimize)


# ------------------------------------------------------------------
# 4.  constraints
# ------------------------------------------------------------------
# u_j − Σ_{i∈𝒫_{jk}} y_ij  ≤ 0,           ∀j∈𝒟, ∀k∈𝒩⁻(j)
def bom_production_rule(mdl, j, k):
    rhs = sum(mdl.y[i, j] for i in data['P'][(j, k)])
    return mdl.u[j] - rhs <= 0
m.BomProduction = Constraint([(j, k) for j in m.D for k in m.N_minus[j]], rule=bom_production_rule)

# Σ_{j∈𝒩⁺(i)} y_ij − u_i ≤ s_i,            ∀ i∈𝒰
def flow_balance_rule(mdl, i):
    return sum(mdl.y[i, j] for j in mdl.N_plus[i]) - mdl.u[i] <= mdl.s[i]
m.FlowBalance = Constraint(m.U, rule=flow_balance_rule)

# u_j = 0,                                 ∀ j∈𝒮⁽ⁿ⁾
m.Disrupted = Constraint(m.S, rule=lambda m, j: m.u[j] == 0)

# l_j + u_j + s_j ≥ d_j · t⁽ⁿ⁾,            ∀ j∈𝒱
def demand_rule(mdl, j):
    return mdl.l[j] + mdl.u[j] + mdl.s[j] >= mdl.d[j] * mdl.t
m.Demand = Constraint(m.V, rule=demand_rule)

# Σ_{k∈𝒜_α} u_k ≤ c_α · t⁽ⁿ⁾,                ∀ j∈NODES
def capacity_rule(mdl, j):
    return mdl.u[j] <= mdl.c[j] * mdl.t
m.Capacity = Constraint(m.NODES, rule=capacity_rule)


# ------------------------------------------------------------------
# 5.  solve
# ------------------------------------------------------------------
if __name__ == "__main__":
    # choose any LP/MIP solver that Pyomo can see (CBC, Gurobi, CPLEX, HiGHS, …)
    solver = SolverFactory("cbc")      # just an example
    result = solver.solve(m, tee=True)
    m.display()                        # quick sanity-check of results
